In [2]:
import numpy as np
import os
from keras.models import load_model

from my_data import *
from my_model import *
from my_applications import *
from my_gradcam import *

ModuleNotFoundError: No module named 'cv2'

### Global variables

In [13]:
ROOT_PATH = '/home/hippolyte/Documents/universite/m1/TER/'
DATASET_NAME = 'chest_xray'
DATASET_PATH = ROOT_PATH + 'datasets/' + DATASET_NAME + '/'
ARRAY_PATH = ROOT_PATH + 'arrays/' + DATASET_NAME + '/'
MODEL_PATH = ROOT_PATH + 'models/' + DATASET_NAME + '/'

HEIGHT, WIDTH, CHANNELS = 224, 224, 3

### Get the data class helper 

In [14]:
dataclass = data_helper(DATASET_NAME,
                        DATASET_PATH,
                        ARRAY_PATH,
                        height=HEIGHT,
                        width=WIDTH,
                        channels=CHANNELS,
                        histogram_equalization=False)

print(dataclass.directories)
print(dataclass.labels)

['test', 'train', 'val']
{'NORMAL': 0, 'PNEUMONIA': 1}


### Load the data

In [15]:
dataclass._get_images_dir('test')

test_x  = dataclass.x['test']
test_y  = dataclass.y['test']

test: getting images from /home/hippolyte/Documents/universite/m1/TER/arrays/chest_xray/test(224,224,3).h5
Data shape : (624, 224, 224, 3)
Labels shape : (624, 2)
Found 624 images in 2 seconds.


### Show images

In [16]:
#dataclass.show_image('train', 0)

## Load model and predict

In [17]:
from keras import applications
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense, InputLayer
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator


import pickle
import numpy as np
from keras.models import Model
from sklearn.metrics import classification_report, confusion_matrix
from matplotlib import pyplot as plt
from mlxtend.plotting import plot_confusion_matrix

In [18]:
def build_model(image_shape,
                classes,
                num_layers_to_freeze=21,
                base_model_weights_path=None,
                top_model_weights_path=None):
    
    model_name = f'vgg19_{num_layers_to_freeze}'
    
    # build the base model
    if base_model_weights_path is None:
        base_model_weights_path = 'imagenet'
    base_model = applications.vgg19.VGG19(input_shape=image_shape,
                                          weights=base_model_weights_path,
                                          include_top=False)
    print('Load VGG19 as base model')
    
    # load base model weights
    #base_model.load_weights(base_model_weights_path)
    
    # adding classification block on top of base model
    x = Flatten(input_shape=image_shape, name='flatten')(base_model.output)
    x = Dense(256, activation='relu', name='fc1')(x)
    x = Dropout(0.5, name='dropout1')(x)
    #x = Dense(512, activation='relu', name='fc2')(x)
    #x = Dropout(0.5, name='dropout2')(x)
    x = Dense(classes, activation='softmax', name='predictions')(x)
    
    # combine both
    model = Model(inputs=base_model.input, outputs=x, name=model_name)
    
    # load top model weights
    #model.load_weights(top_model_weights_path)
    
    # freeze layers
    for i, layer in enumerate(model.layers):
        if i <= num_layers_to_freeze:
            layer.trainable = False
        else:
            layer.trainable = True
            print(f'Layer {i} {layer.name} is trainable')

    # compile model
    loss_type = 'binary_' if classes == 2 else 'categorical_'
    loss_type += 'crossentropy'
    model.compile(loss=loss_type,
                  optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
                  metrics=['accuracy'])
    
    print('Model compiled')
    return model

In [10]:
save_model_path = MODEL_PATH + 'vgg19_aug_256.model'
save_history_path = MODEL_PATH + 'vgg19_aug_256.history'
#BASE_MODEL_WEIGHTS_PATH = ROOT_PATH + 'models/vgg19_weights_no_top.h5'

model = build_model((HEIGHT, WIDTH, CHANNELS), classes=2) #, BASE_MODEL_WEIGHTS_PATH)
model.load_weights(save_model_path)

Instructions for updating:
Colocations handled automatically by placer.
Load VGG19 as base model
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Layer 22 flatten is trainable
Layer 23 fc1 is trainable
Layer 24 dropout1 is trainable
Layer 25 predictions is trainable
Model compiled


OSError: Unable to open file (unable to open file: name = '/home/hippolyte/Documents/universite/m1/TER/models/chest_xray/vgg19_aug_256.model', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

## Check metrics

In [ ]:
preds = np.argmax(model.predict(test_x), axis=-1)
ground_truth = np.argmax(test_y, axis=-1)

In [ ]:
plot_model_performance(preds, ground_truth, dataclass.labels.keys())

In [11]:
plot_model_history(save_history_path)

Loading history from /home/hippolyte/Documents/universite/m1/TER/models/chest_xray/vgg19_aug_256.history


FileNotFoundError: [Errno 2] No such file or directory: '/home/hippolyte/Documents/universite/m1/TER/models/chest_xray/vgg19_aug_256.history'

# Grad CAM

## Let's explore our CNN and look at predictions !

In [ ]:
guided_model = build_guided_model(model)

In [ ]:
def get_label_str_from_code(code):
    for label, _code in dataclass.labels.items():
        if code == _code:
            return label

In [39]:
#print(f'Predicted\n{preds}\nGround Truth\n{ground_truth}')

differences = [0 if p == g else 1 for p, g in zip(preds, ground_truth)]
#print(f'Differences\n{differences}')

print(dict(zip(*np.unique(differences, return_counts=True))))

{0: 568, 1: 56}


In [2]:
img_index = 1

pred_str = get_label_str_from_code(preds[img_index])
true_str = get_label_str_from_code(ground_truth[img_index])

img_test_array = test_x[img_index]

print(f'Explanation for : {pred_str}')
print(f'Ground truth is : {true_str}')

gradcam, gb, guided_gradcam = compute_saliency_array(model,
                                                       guided_model,
                                                       img_test_array,
                                                       HEIGHT,
                                                       WIDTH,
                                                       layer_name='block2_conv2',
                                                       cls=-1,
                                                       visualize=True,
                                                       save=False)

NameError: name 'get_label_str_from_code' is not defined

In [ ]:
"""
#randoms = np.random.choice

for img_index, diffe in enumerate(differences):
    pred_str = get_label_str_from_code(preds[img_index])
    true_str = get_label_str_from_code(ground_truth[img_index])
    
    #if diffe == 1:
    #ind = np.random.randint(len(test_x))
    img_test_array = test_x[img_index]

    print(f'Explanation for : {pred_str}')
    print(f'Ground truth is : {true_str}')

    gradcam, gb, guided_gradcam = compute_saliency_array(model,
                                                           guided_model,
                                                           img_test_array,
                                                           HEIGHT,
                                                           WIDTH,
                                                           layer_name='block5_conv4',
                                                           cls=-1,
                                                           visualize=True,
                                                           save=False)

    num_images_current = num_images_current + 1
    if num_images_current == num_images_to_show:
        break
"""